In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
from netCDF4 import num2date, date2num, date2index
import datetime as dt
import pandas as pd
from StringIO import StringIO
import xarray as xr

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [28]:
nameCSV = 'innisfail_file.csv'
df = pd.read_csv(str(nameCSV), sep=r'\s+', engine='c', header=0, na_filter=False, \
                               dtype=np.float, low_memory=False)

data = df.sort_values(by=['tt'])

lat = data.values[:,0]
lon = data.values[:,1]
wh = data.values[:,2]
tt = data.values[:,3]
qc = data.values[:,4]

In [77]:
netCDF4.num2date(tt[0:200],u'days since 1985-01-01 00:00:00 UTC')

array([datetime.datetime(1985, 3, 31, 8, 13, 20, 375002),
       datetime.datetime(1985, 3, 31, 8, 13, 21, 351561),
       datetime.datetime(1985, 3, 31, 8, 13, 22, 335935),
       datetime.datetime(1985, 3, 31, 8, 13, 23, 312504),
       datetime.datetime(1985, 3, 31, 8, 13, 24, 296878),
       datetime.datetime(1985, 3, 31, 8, 13, 25, 273437),
       datetime.datetime(1985, 3, 31, 8, 13, 26, 257811),
       datetime.datetime(1985, 3, 31, 8, 13, 27, 234380),
       datetime.datetime(1985, 3, 31, 8, 13, 31, 156245),
       datetime.datetime(1985, 3, 31, 8, 13, 32, 132814),
       datetime.datetime(1985, 3, 31, 8, 13, 33, 117188),
       datetime.datetime(1985, 3, 31, 8, 13, 34, 93747),
       datetime.datetime(1985, 3, 31, 8, 13, 35, 70316),
       datetime.datetime(1985, 3, 31, 8, 13, 36, 54690),
       datetime.datetime(1985, 3, 31, 8, 13, 37, 31249),
       datetime.datetime(1985, 3, 31, 8, 13, 38, 15623),
       datetime.datetime(1985, 3, 31, 8, 13, 38, 992192),
       datetime.dat

In [ ]:
139403/20

In [3]:
len(tt)

139403

In [47]:
#TEST TO SEE ALL HIGH VALUES
idd = np.where(np.logical_and(wh>12, wh<13))[0]

for k in range(len(idd)):
    print wh[idd[k]], netCDF4.num2date(tt[idd[k]],u'days since 1985-01-01 00:00:00 UTC')

12.359000205993652 2013-05-18 20:21:41.500005
12.40000057220459 2014-04-12 14:16:28.109379
12.688000679016113 2014-04-12 14:16:35.249996
12.646000862121582 2017-01-14 11:20:25.585933
12.178000450134276 2018-02-16 21:52:04.000004
12.976000785827635 2018-02-16 21:52:07.765624


In [10]:
tt1 = netCDF4.num2date(tt,u'days since 1985-01-01 00:00:00 UTC')
print tt1[0], wh[0]
print netCDF4.num2date(max(tt),u'days since 1985-01-01 00:00:00 UTC')

2016-10-17 05:33:22.656250 1.0600000619888306
2019-02-20 12:17:03.000003


In [33]:
#MAKE MEAN SATELLITE TRACK. DOESNT SEEM TO WORK??
days = [] #daystart
for k in range(len(tt)):
       
    t1 = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
    if k == 0: #get day 0. Then else: every day after that
        days.append(0)
        dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
        it = 0
    else:
        if t1.day != dd.day: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            days.append(k)
            it += 1
            dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')

In [72]:
print netCDF4.num2date(tt[days[0]],u'days since 1985-01-01 00:00:00 UTC')#same as first record

# meanwh is the mean wh of each day (satellite track)
meanwh = []

for k in range(len(days)-1):
    meanwh.append(np.mean(wh[days[k]:days[k+1]])) 
    
print np.mean(meanwh), 'mean from each satellite track'
print np.mean(wh), 'mean from each record'
print len(wh), 'number of records'
print len(meanwh), 'number of satellite tracks'

#slightly different mean wh depending on if done by day, or individual records (each second)

1985-03-31 08:13:20.375002
1.4414013196123379 mean from each satellite track
1.4463256999402663 mean from each record
139403 number of records
3761 number of satellite tracks


In [70]:
netCDF4.num2date(days,u'days since 1985-01-01 00:00:00 UTC')

array([real_datetime(1985, 1, 1, 0, 0), real_datetime(1985, 2, 3, 0, 0),
       real_datetime(1985, 2, 20, 0, 0), ...,
       real_datetime(2366, 1, 16, 0, 0), real_datetime(2366, 3, 9, 0, 0),
       real_datetime(2366, 5, 23, 0, 0)], dtype=object)

In [75]:
139403/20

6970

In [49]:
#why does this give me different mean wh value?

wh_ = wh[days]
qc_ = qc[days]
lon_ = lon[days]
tt_ = tt[days]
lat_ = lat[days]

print max(wh_)
print min(wh_)
print len(tt)
print len(days)
print np.mean(wh_)

24.345001220703125
0.0
139403
3762
1.5149176677001728


In [54]:
#for k in range(len(days)):
#    if wh_[k]>5:
#        print wh_[k], netCDF4.num2date(tt_[k],u'days since 1985-01-01 00:00:00 UTC')

In [50]:
#This should be average per track, wh above 7.

for k in range(len(wh_)):
    if wh_[k]>7:
        print wh_[k], netCDF4.num2date(tt_[k],u'days since 1985-01-01 00:00:00 UTC')

13.909000396728516 2010-11-03 09:53:04.562499
24.345001220703125 2013-02-18 14:34:14.718752
7.449000358581543 2018-02-04 20:13:31.281250
22.916000366210934 2019-02-14 20:13:04.171871
20.75900077819824 2019-02-17 07:54:21.453127


In [51]:
daywh = [] #daystart
for k in range(len(tt)):
    if wh[k]<10:
        t1 = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
        if k == 0: #get day 0. Then else: every day after that
            daywh.append(0)
            dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
            it = 0
        else:
            if t1.day != dd.day: #if day 1 is not equal to day 2, then append
                #print dd.day,t1.day
                daywh.append(k)
                it += 1
                dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')

In [52]:
len(daywh)

3760

In [53]:
wh2 = wh[daywh]
qc2 = qc[daywh]
lon2 = lon[daywh]
tt2 = tt[daywh]
lat2 = lat[daywh]

print max(wh2)
print min(wh2)
print len(tt2)
print np.mean(wh2)

7.449000358581543
0.0
3760
1.4956971444788965


In [ ]:
for k in range(len(wh2)):
    if wh2[k]>7:
        print wh2[k], netCDF4.num2date(tt2[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
netCDF4.num2date(12087.8427231626124,u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
print np.mean(wh2), 'mean'
print max(wh2), 'max'
print min(wh2), 'min'

In [ ]:
 for p in range(len(timing)):
            #print(u,' - ',k,'/',len(urlON),' - ',p,'/',len(timing))
            if timing[p] >= start_date and timing[p] <= end_date:
                if lats[p]>latmin and lats[p]<latmax and lons[p]>lonmin and lons[p]<lonmax:
                    if np.where(np.logical_and(0<wh[p]<10,qc[p]==1)):
                        boxLat.append(lats[p])